In [1]:
pip install pyspark

     |████████████████████████████████| 281.4 MB 33 kB/s 
     |████████████████████████████████| 198 kB 63.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=c7b1c49dd6c61cd9cf716eea6ca2625de0386aeb9536ba7a49682fd33598230d
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
import pyspark as ps
from pyspark.ml.feature import StopWordsRemover
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [23]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
twitter_file="/content/gdrive/MyDrive/sentiment140.csv"

In [5]:
conf = ps.SparkConf().setAll([('spark.executor.memory', '16g'), ('spark.driver.memory', '16g')])
#sc.stop()
sc = ps.SparkContext(conf=conf)

In [6]:
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
sql_context = SQLContext(sc)




/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [9]:
twitterDf = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load(twitter_file)

twitterDf.show()

+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  0|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  0|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  0|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  0|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  0|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  0|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [10]:
from pyspark.sql.functions import regexp_replace

twitterDf = twitterDf.withColumn("_c5", regexp_replace("_c5", "@[A-Za-z0-9]+", ""))
twitterDf = twitterDf.withColumn("_c5", regexp_replace("_c5", "(?:\@|http?\://|https?\://|www)\S+", ""))
#tw = tw.withColumn("Tweet", regexp_replace("Tweet", "#[A-Za-z0-9]+", ""))
twitterDf = twitterDf.withColumn("_c5", regexp_replace("_c5", "'", ""))
twitterDf = twitterDf.withColumn("_c5", regexp_replace("_c5", "[^A-Za-z0-9]+", " "))
twitterDf = twitterDf.withColumn("_c5", f.lower(f.col("_c5")) )

In [11]:
twitterDf.tail(5)

[Row(_c0='4', _c1='2193601966', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='AmandaMarie1028', _c5='just woke up having no school is the best feeling ever '),
 Row(_c0='4', _c1='2193601969', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='TheWDBoards', _c5='thewdb com very cool to hear old walt interviews '),
 Row(_c0='4', _c1='2193601991', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='bpbabe', _c5='are you ready for your mojo makeover ask me for details '),
 Row(_c0='4', _c1='2193602064', _c2='Tue Jun 16 08:40:49 PDT 2009', _c3='NO_QUERY', _c4='tinydiamondz', _c5='happy 38th birthday to my boo of alll time tupac amaru shakur '),
 Row(_c0='4', _c1='2193602129', _c2='Tue Jun 16 08:40:50 PDT 2009', _c3='NO_QUERY', _c4='RyanTrevMorris', _c5='happy charitytuesday ')]

In [12]:
stopwordList = nltk.corpus.stopwords.words('english')

In [13]:
from pyspark.sql.types import DoubleType

twitterDf = twitterDf.withColumn("_c0", twitterDf["_c0"].cast(DoubleType()))

In [14]:

trainDF, testDF = twitterDf.randomSplit([0.80, 0.20], seed=2000)

In [15]:
testDF.show(5,truncate=False)

+---+----------+----------------------------+--------+---------------+----------------------------------------------------------------------------------------+
|_c0|_c1       |_c2                         |_c3     |_c4            |_c5                                                                                     |
+---+----------+----------------------------+--------+---------------+----------------------------------------------------------------------------------------+
|0.0|1467812723|Mon Apr 06 22:20:19 PDT 2009|NO_QUERY|TLeC           | i couldnt bear to watch it and i thought the ua loss was embarrassing                  |
|0.0|1467812771|Mon Apr 06 22:20:19 PDT 2009|NO_QUERY|robrobbierobert| it it counts idk why i did either you never talk to me anymore                         |
|0.0|1467812799|Mon Apr 06 22:20:20 PDT 2009|NO_QUERY|HairByJess     | i wish i got to watch it with you i miss you and how was the premiere                  |
|0.0|1467814119|Mon Apr 06 22:20:40 PDT 

In [16]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer, CountVectorizer
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [17]:
tokenizer = Tokenizer(inputCol="_c5", outputCol="words")
stopwordremover=StopWordsRemover(inputCol="words", outputCol="TweetWithStopwordsfree")
hashtf = HashingTF(numFeatures=2**16, inputCol="TweetWithStopwordsfree", outputCol='tf')
idf = IDF(inputCol='tf', outputCol="features", minDocFreq=5) #minDocFreq: remove sparse terms
#label_stringIdx = StringIndexer(inputCol = "target", outputCol = "label")
pipeline = Pipeline(stages=[tokenizer,stopwordremover, hashtf, idf])

pipelineFit = pipeline.fit(twitterDf)


In [18]:
trainDF = pipelineFit.transform(trainDF)
testDF = pipelineFit.transform(testDF)
trainDF.show(5)

+---+----------+--------------------+--------+---------------+--------------------+--------------------+----------------------+--------------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|               words|TweetWithStopwordsfree|                  tf|            features|
+---+----------+--------------------+--------+---------------+--------------------+--------------------+----------------------+--------------------+--------------------+
|0.0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_| a thats a bummer...|[, a, thats, a, b...|  [, thats, bummer,...|(65536,[10345,216...|(65536,[10345,216...|
|0.0|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|[is, upset, that,...|  [upset, cant, upd...|(65536,[16064,193...|(65536,[16064,193...|
|0.0|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus| i dived many tim...|[, i, dived, many...|  [, dived, many, t...|(65536,[2548,2888...|(6

In [19]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=100)
trainDF=trainDF.selectExpr("_c0 as label", "_c5 as text","words","TweetWithStopwordsfree", "tf","features")
lrModel = lr.fit(trainDF)
testDF=testDF.selectExpr("_c0 as label", "_c5 as text","words","TweetWithStopwordsfree", "tf","features")
predictions = lrModel.transform(testDF)

In [20]:
predictions.show(5)

+-----+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|                text|               words|TweetWithStopwordsfree|                  tf|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  0.0| i couldnt bear t...|[, i, couldnt, be...|  [, couldnt, bear,...|(65536,[513,8997,...|(65536,[513,8997,...|[9.30412012010187...|[0.92843932358713...|       0.0|
|  0.0| it it counts idk...|[, it, it, counts...|  [, counts, idk, e...|(65536,[6589,1383...|(65536,[6589,1383...|[9.16107302381454...|[0.90692659025276...|       0.0|
|  0.0| i wish i got to ...|[, i, wish, i, go...|  [, wish, got, wat...|(65536,[20719,292...|(65536,[20719,292...|[9.62126269400567...|[0.96072507588899...|    

In [21]:

from pyspark.ml.evaluation import MulticlassClassificationEvaluator


evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(predictions)

0.7703392688829602

In [24]:
tweets1 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load('/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/categoryPredictedDataset/catpredicted.csv')
tweets2 = sql_context.read.format('com.databricks.spark.csv').option('header','false').option("delimiter", ",").load('/content/gdrive/MyDrive/Colab Notebooks/BigDataProject/categoryPredictedDataset/catpredicted2.csv')

In [25]:
tweets1.show()
tweets2.show()

+--------------------+-------------+
|                 _c0|          _c1|
+--------------------+-------------+
|gonna try this mi...|        sport|
|the older we get ...|         tech|
|lawyer for journa...|    political|
|     this is so cute|     business|
|yeah what happene...|entertainment|
|      good for you x|entertainment|
|just sold get you...|         tech|
|us health officia...|    political|
|       1990 with him|entertainment|
|not sure this is ...|         tech|
|feels like we wen...|        sport|
|            twittodz|     business|
|my dog and i look...|entertainment|
|blew this into sh...|entertainment|
|these communists ...|        sport|
|so excited to get...|         tech|
|   himeru looks at u|entertainment|
|i t s b e e n a q...|        sport|
|samsung galaxy s2...|         tech|
|this man will joi...|entertainment|
+--------------------+-------------+
only showing top 20 rows

+--------------------+-------------+
|                 _c0|          _c1|
+-----------

In [26]:
import functools
  
def unionAll(dfs):
    return functools.reduce(lambda df1, df2: df1.union(df2.select(df1.columns)), dfs)

In [27]:
tweets = unionAll([tweets2, tweets1])
print(tweets.count())

71730


In [28]:
tweets.show()

+--------------------+-------------+
|                 _c0|          _c1|
+--------------------+-------------+
|how does the djs ...|entertainment|
|imagine you ina s...|entertainment|
|after 5 weeks bas...|entertainment|
|alpha i would lov...|         tech|
|today s meditatio...|entertainment|
|03146 i clearly h...|    political|
|    morning sex any1|entertainment|
|good bye to good ...|entertainment|
|lmfao right like ...|    political|
|you guys are down...|entertainment|
|hurry up to get y...|         tech|
|some nightfood pa...|     business|
|1 clay did not ag...|    political|
|just as always ev...|entertainment|
|and we ll be tuni...|entertainment|
|life is a gift pr...|         tech|
|sell holo shaq en...|     business|
|12 some kids star...|         tech|
|gaethje quickly b...|         tech|
|hey au your inter...|     business|
+--------------------+-------------+
only showing top 20 rows



In [29]:
tweets = tweets.selectExpr("_c0 as _c5","_c1 as catlabel")

In [30]:
tweets.show()

+--------------------+-------------+
|                 _c5|     catlabel|
+--------------------+-------------+
|how does the djs ...|entertainment|
|imagine you ina s...|entertainment|
|after 5 weeks bas...|entertainment|
|alpha i would lov...|         tech|
|today s meditatio...|entertainment|
|03146 i clearly h...|    political|
|    morning sex any1|entertainment|
|good bye to good ...|entertainment|
|lmfao right like ...|    political|
|you guys are down...|entertainment|
|hurry up to get y...|         tech|
|some nightfood pa...|     business|
|1 clay did not ag...|    political|
|just as always ev...|entertainment|
|and we ll be tuni...|entertainment|
|life is a gift pr...|         tech|
|sell holo shaq en...|     business|
|12 some kids star...|         tech|
|gaethje quickly b...|         tech|
|hey au your inter...|     business|
+--------------------+-------------+
only showing top 20 rows



In [31]:
sentimantpredictions = pipelineFit.transform(tweets)

In [32]:
sentimantpredictions = lrModel.transform(sentimantpredictions)

In [33]:
sentimantpredictions.show()

+--------------------+-------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------+
|                 _c5|     catlabel|               words|TweetWithStopwordsfree|                  tf|            features|       rawPrediction|         probability|prediction|
+--------------------+-------------+--------------------+----------------------+--------------------+--------------------+--------------------+--------------------+----------+
|how does the djs ...|entertainment|[how, does, the, ...|  [djs, know, mix, ...|(65536,[9859,1951...|(65536,[9859,1951...|[6.46699089260116...|[0.04264810985613...|       4.0|
|imagine you ina s...|entertainment|[imagine, you, in...|  [imagine, ina, sh...|(65536,[1585,2239...|(65536,[1585,2239...|[7.68345257597664...|[0.33663914607310...|       4.0|
|after 5 weeks bas...|entertainment|[after, 5, weeks,...|  [5, weeks, based,...|(65536,[329,5104,...|(65536,[329,5104,..

In [34]:
sentimantpredictions=sentimantpredictions.drop("words","TweetWithStopwordsfree","tf","features","rawPrediction","probability")

In [35]:
sentimantpredictions.show()

+--------------------+-------------+----------+
|                 _c5|     catlabel|prediction|
+--------------------+-------------+----------+
|how does the djs ...|entertainment|       4.0|
|imagine you ina s...|entertainment|       4.0|
|after 5 weeks bas...|entertainment|       0.0|
|alpha i would lov...|         tech|       4.0|
|today s meditatio...|entertainment|       4.0|
|03146 i clearly h...|    political|       0.0|
|    morning sex any1|entertainment|       4.0|
|good bye to good ...|entertainment|       0.0|
|lmfao right like ...|    political|       4.0|
|you guys are down...|entertainment|       0.0|
|hurry up to get y...|         tech|       4.0|
|some nightfood pa...|     business|       4.0|
|1 clay did not ag...|    political|       4.0|
|just as always ev...|entertainment|       4.0|
|and we ll be tuni...|entertainment|       4.0|
|life is a gift pr...|         tech|       4.0|
|sell holo shaq en...|     business|       0.0|
|12 some kids star...|         tech|    

In [36]:
sentimantpredictions.write.csv("/content/gdrive/MyDrive/Colab Notebooks/finalprediction")